In [2]:
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base, Session

In [3]:
DB_PATH = 'sqlite:///model3.sqlite3'
Base = declarative_base()
ENGINE = sa.create_engine(DB_PATH)

In [4]:
class Model_Base(Base):
    __tablename__ = 'Students'
    uid = sa.Column(sa.Integer(), nullable=False, unique=True, primary_key=True, autoincrement=True)
    date = sa.Column(sa.String())
    tg_id = sa.Column(sa.Text())
    timezone = sa.Column(sa.String())
    stack = sa.Column(sa.Text())
    occupation = sa.Column(sa.Text())
    role = sa.Column(sa.Text())
    project_role = sa.Column(sa.Text())
    weekload = sa.Column(sa.Text())
    course = sa.Column(sa.Text())
    course_time = sa.Column(sa.Text())
    notes = sa.Column(sa.Text())
    pl = sa.Column(sa.String())
    chat = sa.Column(sa.String())
    exited = sa.Column(sa.Text())
    prediction = sa.Column(sa.Float())


In [5]:
Base.metadata.create_all(ENGINE)

In [6]:
old_cols = [
    'Отметка времени',
    'Профиль телеграмм (в формате @username)',
    'Часовой пояс',
    'Стек технологий',
    'По какой специальности SF хотите заявиться?',
    'Роль',
    'В какой роли видите себя в проекте?',
    'Сколько часов в неделю готовы уделять проекту?',
    'Какие другие курсы закончили или находитесь в процессе обучения?',
    'Как долго учитесь на курсах?',
    'Notes',
    'ЯП',
    'Вступление в чат практики.',
    'Выбыл'
]
new_cols = [
    'date',
    'tg_id',
    'timezone',
    'stack',
    'occupation',
    'role',
    'project_role',
    'weekload',
    'course',
    'course_time',
    'notes',
    'pl',
    'chat',
    'exited'
]


In [8]:
import uuid
import pandas as pd

In [13]:
# ini_df = pd.read_csv('data/anketa_ini.csv')
ini_df = pd.read_csv('data/anketa_ini.csv', sep=';', encoding='utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 0: invalid continuation byte

In [43]:
def map_names(old_list: list, new_list: list) -> dict:
    dict_ = dict()
    if len(old_list) == len(new_list):
        for old_name, new_name in zip(old_list, new_list):
            dict_[old_name] = new_name
    return dict_

ini_df.rename(columns=map_names(old_cols, new_cols), inplace=True)
ini_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          320 non-null    object
 1   tg_id         322 non-null    object
 2   timezone      322 non-null    object
 3   stack         321 non-null    object
 4   occupation    322 non-null    object
 5   role          322 non-null    object
 6   project_role  322 non-null    object
 7   weekload      322 non-null    object
 8   course        322 non-null    object
 9   course_time   322 non-null    object
 10  notes         99 non-null     object
 11  pl            205 non-null    object
 12  chat          322 non-null    object
 13  exited        63 non-null     object
dtypes: object(14)
memory usage: 35.3+ KB


In [44]:
from random import random
ini_df['prediction'] = round(random(), 2)
ini_df['prediction'] = ini_df['prediction'].apply(lambda x: round(random(), 2))
display(ini_df.prediction)

0      0.67
1      0.16
2      0.47
3      0.69
4      0.33
       ... 
317    0.80
318    0.42
319    0.73
320    0.85
321    0.35
Name: prediction, Length: 322, dtype: float64

In [45]:
with Session(ENGINE) as s_:
    for index_ in range(ini_df.shape[0]):
        tg_id_ = ini_df.tg_id.iloc[index_]
        project_role_ = ini_df.project_role.iloc[index_]
        prediction_ = ini_df.prediction.iloc[index_]
        new_line_ = Model_Base(tg_id=tg_id_,
                               project_role=project_role_,
                               prediction=prediction_
                              )
        s_.add(new_line_)
    s_.commit()


In [47]:
output_list = []
with Session(ENGINE) as t_:
    all_predictions = t_.query(Model_Base).all()
for student in all_predictions:
    output_list.append({'tg_id': student.tg_id,
                        'project_role': student.project_role,
                        'prediction': student.prediction
                      })
##### print(output_list)